In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from typing import Dict, List, Optional, Tuple, Union
from functools import partial
from tempfile import gettempdir
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from prettytable import PrettyTable
from pathlib import Path
import os
import cv2

#-----------------l5kit lib ------------------------------------------
from l5kit.configs import load_config_data
from l5kit.data import (
    LocalDataManager, ChunkedDataset, 
    get_agents_slice_from_frames, get_frames_slice_from_scenes,get_tl_faces_slice_from_frames,
    filter_agents_by_labels, filter_tl_faces_by_frames
)
from l5kit.data.labels import PERCEPTION_LABEL_TO_INDEX 
from l5kit.data.filter import filter_agents_by_labels, filter_agents_by_track_id, filter_agents_by_frames
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import (
    build_rasterizer, Rasterizer, RenderContext,
    EGO_EXTENT_HEIGHT, EGO_EXTENT_LENGTH, EGO_EXTENT_WIDTH
)
from l5kit.rasterization.semantic_rasterizer import CV2_SHIFT, cv2_subpixel
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points, angular_distance, compute_agent_pose, rotation33_as_yaw,transform_point
from l5kit.geometry.transform import yaw_as_rotation33
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from l5kit.kinematic import Perturbation
from l5kit.sampling.slicing import get_future_slice, get_history_slice
from l5kit.data.zarr_dataset import AGENT_DTYPE

#------------ Tensorflow lib ---------------------------------------------
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
from tensorflow.data import Dataset
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications import EfficientNetB0

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
cfg = {
    'format_version': 4,
    'model_params': {
        'model_architecture': "EfficientNetB0"
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1
    },
    
    'raster_params': {
        'raster_size': [224, 224],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5,
        'disable_traffic_light_faces': False
    },
    
    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 12,
        'shuffle': True,
        'num_workers': 4
    }
}

In [ ]:
DIR_INPUT = "/kaggle/input/lyft-motion-prediction-autonomous-vehicles"
MIN_FRAME_HISTORY = 10
MIN_FRAME_FUTURE = 1
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)
raster_w = cfg["raster_params"]["raster_size"][0]
raster_h = cfg["raster_params"]["raster_size"][1]
input_deep = 8
length_of_rnn_sequence = cfg["model_params"]["history_num_frames"] + 1
future_output_shape = (cfg["model_params"]["future_num_frames"], 2)
BATCHSIZE = cfg["train_data_loader"]["batch_size"]
inner_cnn_model = cfg["model_params"]["model_architecture"]

In [ ]:
class RasterExpander():
    def __init__(self, render_context: RenderContext, filter_agents_threshold: float, history_num_frames: int):
        self.render_context = render_context
        self.raster_size = render_context.raster_size_px
        self.filter_agents_threshold = filter_agents_threshold
        self.history_num_frames = history_num_frames
        self.labels = ['PERCEPTION_LABEL_CAR', 'PERCEPTION_LABEL_BICYCLE', 'PERCEPTION_LABEL_PEDESTRIAN']
        self.num_of_layers = len(self.labels) + 2    

    def AddDeepness(self, history_frames: np.ndarray, history_agents: List[np.ndarray], 
                    history_tl_faces: List[np.ndarray], agent: Optional[np.ndarray] = None) -> np.ndarray:
        frame = history_frames[0]
        if agent is None:
            ego_translation_m = history_frames[0]["ego_translation"]
            ego_yaw_rad = rotation33_as_yaw(frame["ego_rotation"])
        else:
            ego_translation_m = np.append(agent["centroid"], history_frames[0]["ego_translation"][-1])
            ego_yaw_rad = agent["yaw"]

        raster_from_world = self.render_context.raster_from_world(ego_translation_m, ego_yaw_rad)
        out_shape = (self.raster_size[1], self.raster_size[0], self.history_num_frames + 1, self.num_of_layers)
        agents_images = np.zeros(out_shape, dtype=np.float32)

        for i, (frame, agents) in enumerate(zip(history_frames, history_agents)):
            agents = filter_agents_by_labels(agents, self.filter_agents_threshold)
            # note the cast is for legacy support of dataset before April 2020
            av_agent = self.get_ego_as_agent(frame).astype(agents.dtype)

            if agent is None:
                agents_image = self.expand_into_deep(self.raster_size, raster_from_world, agents)
            else:
                agent_ego = filter_agents_by_track_id(agents, agent["track_id"])
                if len(agent_ego) != 0:  
                    agents = agents[agents != agent_ego[0]]
                agents_image = self.expand_into_deep(self.raster_size, raster_from_world, np.append(agents, av_agent))
            agents_images[..., i,:self.num_of_layers] = agents_image.transpose((1,2,0))
        return agents_images
    
    def expand_into_deep(self, raster_size: Tuple[int, int], raster_from_world: np.ndarray, agents: np.ndarray) -> np.ndarray:
        layers = np.zeros((self.num_of_layers,raster_size[1], raster_size[0]), dtype=np.float32)
        box_world_coords = np.zeros((len(agents), 4, 2))
        corners_base_coords = np.asarray([[-1, -1], [-1, 1], [1, 1], [1, -1]])

        for idx, agent in enumerate(agents):
            corners = corners_base_coords * agent["extent"][:2] / 2  # corners in zero
            r_m = yaw_as_rotation33(agent["yaw"])
            box_world_coords[idx] = transform_points(corners, r_m) + agent["centroid"][:2]
    
        box_raster_coords = transform_points(box_world_coords.reshape((-1, 2)), raster_from_world)
        box_raster_coords = cv2_subpixel(box_raster_coords.reshape((-1, 4, 2)))
    
        for idx, agent in enumerate(agents):
            #print(agent["label_probabilities"])
            for i, label in enumerate(self.labels):
                lix = PERCEPTION_LABEL_TO_INDEX[label]
                color = float(agent["label_probabilities"][lix])
                cv2.fillPoly(layers[i,...], box_raster_coords[[idx]], color=color, shift=CV2_SHIFT)
            i = len(self.labels)
            cv2.fillPoly(layers[i,...], box_raster_coords[[idx]], color=float(agent["velocity"][0]), shift=CV2_SHIFT)
            cv2.fillPoly(layers[i + 1,...], box_raster_coords[[idx]], color=float(agent["velocity"][1]),shift=CV2_SHIFT)     
        return layers
    
    def get_ego_as_agent(self,frame: np.ndarray) -> np.ndarray:  # TODO this can be useful to have around
        ego_agent = np.zeros(1, dtype=AGENT_DTYPE)
        ego_agent[0]["centroid"] = frame["ego_translation"][:2]
        ego_agent[0]["yaw"] = rotation33_as_yaw(frame["ego_rotation"])
        ego_agent[0]["extent"] = np.asarray((EGO_EXTENT_LENGTH, EGO_EXTENT_WIDTH, EGO_EXTENT_HEIGHT))
        return ego_agent
    
def _create_targets_for_deep_prediction(num_frames: int, frames: np.ndarray, selected_track_id: Optional[int], 
                                        agents: List[np.ndarray], agent_from_world: np.ndarray,current_agent_yaw: float
                                       ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    coords_offset = np.zeros((num_frames, 2), dtype=np.float32)
    yaws_offset = np.zeros((num_frames, 1), dtype=np.float32)
    availability = np.zeros((num_frames,), dtype=np.float32)

    for i, (frame, frame_agents) in enumerate(zip(frames, agents)):
        if selected_track_id is None:
            agent_centroid = frame["ego_translation"][:2]
            agent_yaw = rotation33_as_yaw(frame["ego_rotation"])
        else:
            # it's not guaranteed the target will be in every frame
            try:
                agent = filter_agents_by_track_id(frame_agents, selected_track_id)[0]
                agent_centroid = agent["centroid"]
                agent_yaw = agent["yaw"]
            except IndexError:
                availability[i] = 0.0  # keep track of invalid futures/history
                continue

        coords_offset[i] = transform_point(agent_centroid, agent_from_world)
        yaws_offset[i] = angular_distance(agent_yaw, current_agent_yaw)
        availability[i] = 1.0
    return coords_offset, yaws_offset, availability

def decorated_generate_agent_sample(state_index: int, frames: np.ndarray, agents: np.ndarray, tl_faces: np.ndarray,
                                    selected_track_id: Optional[int], render_context: RenderContext, history_num_frames: int,
                                    history_step_size: int, future_num_frames: int, future_step_size: int,
                                    filter_agents_threshold: float, rasterizer: Rasterizer) -> dict:
    #  the history slice is ordered starting from the latest frame and goes backward in time., ex. slice(100, 91, -2)
    history_slice = get_history_slice(state_index, history_num_frames, history_step_size, include_current_state=True)
    future_slice = get_future_slice(state_index, future_num_frames, future_step_size)
    history_frames = frames[history_slice].copy()  # copy() required if the object is a np.ndarray
    future_frames = frames[future_slice].copy()
    sorted_frames = np.concatenate((history_frames[::-1], future_frames))  # from past to future
    agent_slice = get_agents_slice_from_frames(sorted_frames[0], sorted_frames[-1])
    agents = agents[agent_slice].copy()  # this is the minimum slice of agents we need
    history_frames["agent_index_interval"] -= agent_slice.start  # sync interval with the agents array
    future_frames["agent_index_interval"] -= agent_slice.start  # sync interval with the agents array
    history_agents = filter_agents_by_frames(history_frames, agents)
    future_agents = filter_agents_by_frames(future_frames, agents)
    tl_slice = get_tl_faces_slice_from_frames(history_frames[-1], history_frames[0])  # -1 is the farthest
    # sync interval with the traffic light faces array
    history_frames["traffic_light_faces_index_interval"] -= tl_slice.start
    history_tl_faces = filter_tl_faces_by_frames(history_frames, tl_faces[tl_slice].copy())
    # State you want to predict the future of.
    cur_frame = history_frames[0]
    cur_agents = history_agents[0]

    if selected_track_id is None:
        agent_centroid_m = cur_frame["ego_translation"][:2]
        agent_yaw_rad = rotation33_as_yaw(cur_frame["ego_rotation"])
        agent_extent_m = np.asarray((EGO_EXTENT_LENGTH, EGO_EXTENT_WIDTH, EGO_EXTENT_HEIGHT))
        selected_agent = None
    else:
        try:
            agent = filter_agents_by_track_id(
                filter_agents_by_labels(cur_agents, filter_agents_threshold), selected_track_id
            )[0]
        except IndexError:
            raise ValueError(f" track_id {selected_track_id} not in frame or below threshold")
        agent_centroid_m = agent["centroid"]
        agent_yaw_rad = float(agent["yaw"])
        agent_extent_m = agent["extent"]
        selected_agent = agent

    input_im = (
        None
        if not rasterizer
        else rasterizer.rasterize(history_frames, history_agents, history_tl_faces, selected_agent)
    )
    
    rasterExpander = RasterExpander(rasterizer.render_context, rasterizer.filter_agents_threshold,rasterizer.history_num_frames)
    deepLayers = rasterExpander.AddDeepness(history_frames, history_agents, history_tl_faces, selected_agent)

    world_from_agent = compute_agent_pose(agent_centroid_m, agent_yaw_rad)
    agent_from_world = np.linalg.inv(world_from_agent)
    raster_from_world = render_context.raster_from_world(agent_centroid_m, agent_yaw_rad)

    future_coords_offset, future_yaws_offset, future_availability = _create_targets_for_deep_prediction(
        future_num_frames, future_frames, selected_track_id, future_agents, agent_from_world, agent_yaw_rad
    )

    # history_num_frames + 1 because it also includes the current frame
    history_coords_offset, history_yaws_offset, history_availability = _create_targets_for_deep_prediction(
        history_num_frames + 1, history_frames, selected_track_id, history_agents, agent_from_world, agent_yaw_rad
    )

    return {
        "image": input_im,
        "deep_layers": deepLayers,
        "target_positions": future_coords_offset,
        "target_yaws": future_yaws_offset,
        "target_availabilities": future_availability,
        "history_positions": history_coords_offset,
        "history_yaws": history_yaws_offset,
        "history_availabilities": history_availability,
        "world_to_image": raster_from_world,  # TODO deprecate
        "raster_from_agent": raster_from_world @ world_from_agent,
        "raster_from_world": raster_from_world,
        "agent_from_world": agent_from_world,
        "world_from_agent": world_from_agent,
        "centroid": agent_centroid_m,
        "yaw": agent_yaw_rad,
        "extent": agent_extent_m,
    }

class DecoratedAgentDataset(AgentDataset):
    def __init__(
        self,
        cfg: dict,
        zarr_dataset: ChunkedDataset,
        rasterizer: Rasterizer,
        perturbation: Optional[Perturbation] = None,
        agents_mask: Optional[np.ndarray] = None,
        min_frame_history: int = MIN_FRAME_HISTORY,
        min_frame_future: int = MIN_FRAME_FUTURE,
    ):
        assert perturbation is None, "AgentDataset does not support perturbation (yet)"
        super(DecoratedAgentDataset, self).__init__(cfg, zarr_dataset, rasterizer, perturbation, agents_mask,
                                                    min_frame_history, min_frame_future)
        render_context = RenderContext(
            raster_size_px=np.array(cfg["raster_params"]["raster_size"]),
            pixel_size_m=np.array(cfg["raster_params"]["pixel_size"]),
            center_in_raster_ratio=np.array(cfg["raster_params"]["ego_center"]),
        )
        self.sample_function = partial(
            decorated_generate_agent_sample,
            render_context=render_context,
            history_num_frames=cfg["model_params"]["history_num_frames"],
            history_step_size=cfg["model_params"]["history_step_size"],
            future_num_frames=cfg["model_params"]["future_num_frames"],
            future_step_size=cfg["model_params"]["future_step_size"],
            filter_agents_threshold=cfg["raster_params"]["filter_agents_threshold"],
            rasterizer=rasterizer
        )
        
    def get_frame(self, scene_index: int, state_index: int, track_id: Optional[int] = None) -> dict:
        frames = self.dataset.frames[get_frames_slice_from_scenes(self.dataset.scenes[scene_index])]

        tl_faces = self.dataset.tl_faces
        try:
            if self.cfg["raster_params"]["disable_traffic_light_faces"]:
                tl_faces = np.empty(0, dtype=self.dataset.tl_faces.dtype)  # completely disable traffic light faces
        except KeyError:
            warnings.warn(
                "disable_traffic_light_faces not found in config, this will raise an error in the future",
                RuntimeWarning,
                stacklevel=2,
            )
        data = self.sample_function(state_index, frames, self.dataset.agents, tl_faces, track_id)
        img = data["image"]
        k =cfg["model_params"]["history_num_frames"]
        imgs = [self.rasterizer.to_rgb(img[...,[i, i + k + 1, -3,-2,-1]])[np.newaxis,...] for i in range(k + 1)]
        imgs = np.concatenate(imgs,axis = 0) / 255.
        context_layers = np.concatenate((imgs, data[ "deep_layers"].transpose(2,0,1,3)), axis = -1)
        target_positions = np.array(data["target_positions"], dtype=np.float32)
        timestamp = frames[state_index]["timestamp"]
        track_id = np.int64(-1 if track_id is None else track_id)  # always a number to avoid crashing torch
        return {
            "context_layers" : context_layers,
            "target_positions": target_positions,
            "timestamp": timestamp,
            "track_id" : track_id
        }

In [ ]:
class TrainAgentGenerator():
    def __init__(self, agentdataset, number_of_samples):
        self.agentDataset = agentdataset
        self.number_of_samples = number_of_samples
    
    def __call__(self):
        for _ in range(self.number_of_samples * 10):
            i = np.random.randint(0,self.number_of_samples - 10)
            yield (self.agentDataset[i]["context_layers"], self.agentDataset[i]["target_positions"])

In [ ]:
train_cfg = cfg["train_data_loader"]
rasterizer = build_rasterizer(cfg, dm)
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()

In [ ]:
train_dataset = DecoratedAgentDataset(cfg, train_zarr, rasterizer, min_frame_history = 10, min_frame_future = 50)
Number_of_samples = len(train_dataset)
print(Number_of_samples)
train_gen = TrainAgentGenerator(train_dataset, Number_of_samples)
train_dataset = Dataset.from_generator(train_gen,(tf.float32, tf.float32))

In [ ]:
class AgentRNNCell(tfkl.Layer):
    def __init__(self, model, **kwargs):
        self.model = model
        self.units = model.output.shape[-1]
        self.state_size = self.units
        super(AgentRNNCell, self).__init__(**kwargs)
        self.recurrent_kernel = self.add_weight( shape=(self.units, self.units),initializer='uniform',name='recurrent_kernel')

    def call(self, inputs, states):
        prev_output = states[0]
        h = self.model(inputs)
        output = h + tf.tensordot(prev_output, self.recurrent_kernel,axes = 1)
        return output, [output]

def make_cnn_model(nn):
    for i in range(1,len(nn.layers)):
        nn.layers[i].trainable = True
    inputs = tfkl.Input(shape = (raster_w,raster_h,input_deep))
    x = tfkl.Conv2D(filters = 3, kernel_size = 3, padding = "same")(inputs)
    x = nn(x)
    x = tfkl.Dropout(0.1)(x)
    x = tfkl.Dense(1024, kernel_regularizer=tfk.regularizers.l2(0.0001))(x)
    outputs = tfkl.LeakyReLU(0.1)(x)
    model = tfk.Model(inputs,outputs)
    return model

def make_rnn_model(nn):
    model_cnn = make_cnn_model(nn)
    cell = AgentRNNCell(model_cnn)
    inputs = tfkl.Input(shape = (length_of_rnn_sequence, raster_w,raster_h,input_deep))
    x = tfkl.RNN(cell)(inputs)
    x = tfkl.Dense(future_output_shape[0] * future_output_shape[1], kernel_regularizer=tfk.regularizers.l2(0.0001))(x)
    outputs = tfkl.Reshape(future_output_shape)(x)
    model = tfk.Model(inputs,outputs)
    return model

In [ ]:
predef_models = {'MobileNet' : MobileNet(include_top=False, pooling = "avg"),
                'EfficientNetB0' : EfficientNetB0(include_top=False, input_shape=(224,224,3),pooling="avg")}

In [ ]:
weight_filepath = 
model = make_rnn_model(predef_models[inner_cnn_model])
model.compile(optimizer="Adam", loss="mse", metrics=["mae"])
model.load_weights(weight_filepath)

In [ ]:
checkpoint_filepath = './weights_{loss:.2f}_{epoch:02d}.hdf5'
#checkpoint_filepath = './best_weights.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='loss',
    mode='auto',
    save_freq = 200,
    save_best_only=False)

In [ ]:
srcdata = train_dataset.batch(BATCHSIZE).shuffle(True)

In [ ]:
histv = model.fit(srcdata, epochs=1, callbacks=[model_checkpoint_callback])

In [ ]:
#model.save_weights(filepath, overwrite=True, save_format=None, options=None)

In [ ]:
class TrainAgentGenerator():
    def __init__(self, number_of_samples):
        self.number_of_samples = number_of_samples
    
    def __call__(self):
        for _ in range(100):
            i = np.random.randint(0,self.number_of_samples - 10)
            yield i

get = TrainAgentGenerator(7000000)
for i in get():
    print(i)

In [6]:
import os

In [7]:
os.environ

environ{'ALLUSERSPROFILE': 'C:\\ProgramData',
        'APPDATA': 'C:\\Users\\PLDD\\AppData\\Roaming',
        'ASL.LOG': 'Destination=file',
        'BITS': '64',
        'CC': 'cl.exe',
        'CHOCOLATEYINSTALL': 'C:\\ProgramData\\chocolatey',
        'CHOCOLATEYLASTPATHUPDATE': '132305661523328394',
        'CMAKE_GEN': 'Visual Studio 15 2017 Win64',
        'CMAKE_GENERATOR': 'Visual Studio 15 2017 Win64',
        'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files',
        'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files',
        'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files',
        'COMPUTERNAME': 'DESKTOP-KRAU383',
        'COMSPEC': 'C:\\WINDOWS\\system32\\cmd.exe',
        'CONDA_DEFAULT_ENV': 'tf',
        'CONDA_EXE': 'C:\\Users\\PLDD\\Anaconda3\\Scripts\\conda.exe',
        'CONDA_PREFIX': 'C:\\Users\\PLDD\\Anaconda3\\envs\\tf',
        'CONDA_PREFIX_1': 'C:\\Users\\PLDD\\Anaconda3',
        'CONDA_PROMPT_MODIFIER': '(tf) ',
        'CONDA_

In [12]:
import tensorflow as tf

import time


In [13]:
class ArtificialDataset(tf.data.Dataset):
    def _generator(num_samples):
        # Opening the file
        time.sleep(0.03)

        for sample_idx in range(num_samples):
            # Reading data (line, record) from the file
            time.sleep(0.015)

            yield (sample_idx,)

    def __new__(cls, num_samples=3):
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=tf.dtypes.int64,
            output_shapes=(1,),
            args=(num_samples,)
        )

In [15]:
for l in ArtificialDataset():
    print(l)

tf.Tensor([0], shape=(1,), dtype=int64)
tf.Tensor([1], shape=(1,), dtype=int64)
tf.Tensor([2], shape=(1,), dtype=int64)


In [23]:
d1 = ArtificialDataset()
print(d1)
d2 = d1
print(d2)

<FlatMapDataset shapes: (1,), types: tf.int64>
<FlatMapDataset shapes: (1,), types: tf.int64>


In [31]:
def benchmark(dataset, num_epochs=2):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performing a training step
            time.sleep(0.01)
    tf.print("Execution time:", time.perf_counter() - start_time)

In [39]:
benchmark(ArtificialDataset())

Execution time: 0.28168459999142215


In [40]:
benchmark(
    ArtificialDataset()
    .prefetch(tf.data.experimental.AUTOTUNE)
)

Execution time: 0.22297770000295714


In [42]:
benchmark(
    tf.data.Dataset.range(2)
    .interleave(
        ArtificialDataset,
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
)

Execution time: 0.1901561000267975


In [43]:
benchmark(
    tf.data.Dataset.range(2)
    .interleave(ArtificialDataset)
)

Execution time: 0.2692912999773398


In [3]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as tfkl

In [11]:
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=3, strides=2, padding="valid", input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))])
model.compile(loss="mse", optimizer="adam")
#history = model.fit(X_train, Y_train[:, 3::2], epochs=20,
#validation_data=(X_valid, Y_valid[:, 3::2]))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, None, 20)          80        
_________________________________________________________________
gru_12 (GRU)                 (None, None, 20)          2520      
_________________________________________________________________
gru_13 (GRU)                 (None, None, 20)          2520      
_________________________________________________________________
time_distributed_6 (TimeDist (None, None, 10)          210       
Total params: 5,330
Trainable params: 5,330
Non-trainable params: 0
_________________________________________________________________
